In [1]:
# https://keras.io/
!pip install -q keras
import keras

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 256
num_classes = 10
epochs = 250
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [7]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [8]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [9]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/250
50000/50000 [==============================] - 77s 2ms/step - loss: 2.3196 - acc: 0.1296 - val_loss: 2.2837 - val_acc: 0.1265
Epoch 2/250
50000/50000 [==============================] - 64s 1ms/step - loss: 2.1877 - acc: 0.1934 - val_loss: 2.2525 - val_acc: 0.1504
Epoch 3/250
50000/50000 [==============================] - 65s 1ms/step - loss: 2.0054 - acc: 0.2536 - val_loss: 2.0258 - val_acc: 0.2349
Epoch 4/250
50000/50000 [==============================] - 65s 1ms/step - loss: 1.8366 - acc: 0.3005 - val_loss: 1.8662 - val_acc: 0.2882
Epoch 5/250
50000/50000 [==============================] - 66s 1ms/step - loss: 1.7507 - acc: 0.3275 - val_loss: 1.8544 - val_acc: 0.3268
Epoch 6/250
50000/50000 [==============================] - 65s 1ms/step - loss: 1.7015 - acc: 0.3425 - val_loss: 1.7477 - val_acc: 0.3476
Epoch 7/250
50000/50000 [==============================] - 66s 1ms/step - loss: 1.6637 - acc: 0.3611 - val_loss: 2.1598 - 

50000/50000 [==============================] - 65s 1ms/step - loss: 1.0637 - acc: 0.6137 - val_loss: 1.3769 - val_acc: 0.5554
Epoch 60/250
50000/50000 [==============================] - 66s 1ms/step - loss: 1.0580 - acc: 0.6152 - val_loss: 1.3360 - val_acc: 0.5570
Epoch 61/250
50000/50000 [==============================] - 66s 1ms/step - loss: 1.0526 - acc: 0.6185 - val_loss: 1.4206 - val_acc: 0.5267
Epoch 62/250
50000/50000 [==============================] - 66s 1ms/step - loss: 1.0465 - acc: 0.6212 - val_loss: 1.1907 - val_acc: 0.5887
Epoch 63/250
50000/50000 [==============================] - 66s 1ms/step - loss: 1.0439 - acc: 0.6211 - val_loss: 1.4075 - val_acc: 0.5476
Epoch 64/250
50000/50000 [==============================] - 66s 1ms/step - loss: 1.0403 - acc: 0.6239 - val_loss: 1.6197 - val_acc: 0.4981
Epoch 65/250
50000/50000 [==============================] - 67s 1ms/step - loss: 1.0313 - acc: 0.6260 - val_loss: 1.1206 - val_acc: 0.6082
Epoch 66/250
50000/50000 [==============

Epoch 118/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8767 - acc: 0.6857 - val_loss: 1.0554 - val_acc: 0.6496
Epoch 119/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8771 - acc: 0.6848 - val_loss: 1.2925 - val_acc: 0.5856
Epoch 120/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8751 - acc: 0.6832 - val_loss: 0.9731 - val_acc: 0.6680
Epoch 121/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8714 - acc: 0.6867 - val_loss: 1.0115 - val_acc: 0.6564
Epoch 122/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8703 - acc: 0.6858 - val_loss: 0.9859 - val_acc: 0.6670
Epoch 123/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8669 - acc: 0.6854 - val_loss: 1.0470 - val_acc: 0.6496
Epoch 124/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8621 - acc: 0.6893 - val_loss: 1.0451 - val_acc: 0.6531
Epoch 125/250
50000/

50000/50000 [==============================] - 66s 1ms/step - loss: 0.7747 - acc: 0.7219 - val_loss: 0.8713 - val_acc: 0.7105
Epoch 177/250
50000/50000 [==============================] - 67s 1ms/step - loss: 0.7697 - acc: 0.7229 - val_loss: 1.0784 - val_acc: 0.6701
Epoch 178/250
50000/50000 [==============================] - 67s 1ms/step - loss: 0.7683 - acc: 0.7257 - val_loss: 0.8747 - val_acc: 0.7016
Epoch 179/250
50000/50000 [==============================] - 67s 1ms/step - loss: 0.7674 - acc: 0.7244 - val_loss: 0.9162 - val_acc: 0.6960
Epoch 180/250
50000/50000 [==============================] - 67s 1ms/step - loss: 0.7665 - acc: 0.7247 - val_loss: 0.8983 - val_acc: 0.6987
Epoch 181/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.7645 - acc: 0.7261 - val_loss: 0.8916 - val_acc: 0.7015
Epoch 182/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.7655 - acc: 0.7245 - val_loss: 1.0548 - val_acc: 0.6705
Epoch 183/250
50000/50000 [=======

50000/50000 [==============================] - 67s 1ms/step - loss: 0.7018 - acc: 0.7502 - val_loss: 0.7997 - val_acc: 0.7344
Epoch 235/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.7015 - acc: 0.7496 - val_loss: 1.0420 - val_acc: 0.6720
Epoch 236/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.7004 - acc: 0.7509 - val_loss: 0.9247 - val_acc: 0.7110
Epoch 237/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.6949 - acc: 0.7524 - val_loss: 0.8387 - val_acc: 0.7221
Epoch 238/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.6919 - acc: 0.7547 - val_loss: 0.8980 - val_acc: 0.7179
Epoch 239/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.6932 - acc: 0.7538 - val_loss: 0.9565 - val_acc: 0.7037
Epoch 240/250
50000/50000 [==============================] - 66s 1ms/step - loss: 0.6936 - acc: 0.7521 - val_loss: 0.8959 - val_acc: 0.7157
Epoch 241/250
50000/50000 [=======

In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 7s 739us/step
Test loss: 0.9452508332252503
Test accuracy: 0.7034


In [14]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_modelv2.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# from google.colab import files

# files.download('DNST_model.h5')